In [ ]:
#%%

from flower_init import *

#%%

xs,ys = flowers_init([100, 100])

#%%

#이미지 증폭 라이브러리
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#%%

#데이터 증폭의 옵션을 결정하는 객체 선언
image_generator = ImageDataGenerator(rotation_range=30,
                                     zoom_range = 0.7,
                                     shear_range=0.75,
                                     width_shift_range=0.45,
                                     height_shift_range=0.45,
                                     horizontal_flip=True,
                                     vertical_flip=True)

#%% md
'''
이미지 데이터의 정규화 과정
왜 정규화 할때 MinMaxScaler 를 사용할 것인가요?
'''
#%%

#
image_generator.flow()


#%%

#정규화
xs_norm = xs - 0 / (255.0 -0)

#%%

import numpy as np
#shuffle
shuffle_map = np.arange(xs_norm.shape[0])
np.random.shuffle((shuffle_map))

#%%

train_ratio = 0.8

test_begin_index = int(xs_norm.shape[0] * train_ratio)

train_x = xs_norm[shuffle_map[:test_begin_index]]
test_x = xs_norm[shuffle_map[test_begin_index:]]


train_y = ys[shuffle_map[:test_begin_index]]
test_y= ys[shuffle_map[test_begin_index:]]

#%%

#하나의 데이터를 100장 증폭 과정
train_x[0].shape

#%%

augment_size = 100
#flow 이미지 데이터 증폭 함수
#정석: np.tile(1차원 데이터를 받음) 실제 : np.tile(3차원 데이터가 들어감
x_augment = image_generator.flow(x = np.tile(train_x[0].reshape(100,100,3),reps = augment_size ).reshape(-1,100,100,3),
                                 y = np.zeros(augment_size),
                                 batch_size=augment_size,
                                 shuffle = False #데이터 증폭시 중복을 허용할거냐 False = 안함
                                 ).next()[0] #증폭된 종속변수 만 가져 올수 있다.
#증폭된 종속 변수만 가져올때는 .next()[0]
#증폭할떄 몇번 반복해야 하는지도 x에 전달 시켜줘야함
#np.tile A = tile 의 대상 /

#%%

x_augment.shape

#%%

import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.axis('off')
    i = np.random.choice(100, replace = False)
    plt.imshow(x_augment[i])
plt.show()

#%%

#1.노이즈 증폭 / 2.GAN 을 사용
# 원본 데이터 증폭 시켜주기 train_x
#원본 데이터 의 50 % 만 해당하는 복제본 생성 한 후에 복사 데이터를 증폭 시켜주는 과정 수행

agument_size = int(train_x.shape[0] * 0.5) #절반만 증폭

print(agument_size)

#%%

#복사본 데이터 생성 과정
x_choice = np.random.choice(train_x.shape[0],size = augment_size,replace=False)
#중복 허용을 금지하고  train_x 의 절반의 데이터 만 가져온다

x_augmented = train_x[x_choice].copy() #랜덤의 받아온 index 를 copy 하여 저장
x_augmented.shape

y_augmented = train_y[x_choice].copy()
y_augmented.shape

#%%

#노이즈를 포함하여 증폭 시킨 x 의 독립변수 데이터
x_augmented = image_generator.flow(x_augmented,
                                   np.zeros(augment_size),
                                   batch_size=augment_size,
                                   shuffle= False).next()[0] #독립변수의 증폭값만 가져옴

#%%

plt.figure(figsize=(30,30))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.axis('off')
    i = np.random.choice(100, replace = False)
    plt.imshow(x_augment[i])
plt.show()

#%%

#원본데이터 trainx,y 랑 증폭을 시킨 데이터를 합쳐주기 concatenate
train_x = np.concatenate((train_x,x_augmented))
train_y = np.concatenate((train_y,y_augmented))

#%%

## 전의 학습
#학습된 layer 만 가져와서 우리의 데이터로 학습하는 방식 출력계층은 다시 재정의 해서 만든다.
#학습이 되어진 데이터를 가져오지만 학습을 더시켜야 한다. 이미 학습이 완료한 계층을 학습을 시키지 않고 Freeze 를 수행한다.
#내가 지금 가지고 있는데이터는 가져온 모델이랑 안맞는게 당연하다 이때 일부만 freeze 를 해체 하여 학습을 더 시킬수 있다.
#데이터의 크기에따라 얼려지는 데이터의 양이 다 다르다.

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import MobileNetV2

#%%

mobile_v2= MobileNetV2()

mobile_v2.summary()

#data 를 받기 2가지
#1그냥 링크 다고 다운 받기

#%%

import tensorflow as tf
tf.keras.utils.get_file('/content/labels.csv','http://bit.ly/2GDxsYS')
tf.keras.utils.get_file('/content/train.zip','http://bit.ly/31nIyel')
tf.keras.utils.get_file('/content/test.zip','http://bit.ly/2GHEsnO')


import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 #openCV -> pip install opencv-python / pip install cv2
import os
import csv

from PIL import Image

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input

mobile_v2 = MobileNetV2()
mobile_v2.summary()

label_text = pd.read_csv('dog-breed-identification/labels.csv')

#견종의 수가 얼마나 되는지 확인
label_text['bread'].nunique() #120 가지의 견종의 내용이 있음


#이미지와 견종의 정보에 대한 출력

# 사진당 크기
plt.figure(figsize=(20,20))

# 총 16장의 사진을 출력
for i in range(16):

    # 가로 4개 세로 4개씩 하여 이미지를 출력합니다.
    plt.subplot(4,4,i+1)

    # 무작위 값을 주어 매번 다른 이미지가 출력되도록 합니다.
    i = np.random.choice(1000)

    #label_text 변수에 loc를 사용하여
    # [i번 째 행,'id' 라는 열] 의 값을 할당합니다.
    image_id = label_text.loc[i, 'id']

    #절대경로 설정을 활용하여 사진 출력
    # 이미지
    plt.imshow(plt.imread('dataset/dogs/train/train/' + image_id + '.jpg'))

    #사진과 함께 번호(i) + 제목 ( loc[i,'breed'] ) 출력
    plt.title(str(i) + '-' + label_text.loc[i, 'breed'])

    # 축 정보 표기 x
    plt.axis('off')

plt.show()

#전의 학습을 위한 두가지 실험
#1.가지고온 신경망의 모든 계층을 초기화  다시 학습을 시키는 과정 (학습 x -> 시간과 성능이 구림)
#2.가지고 온 신경망의 절반의 계층을 초기화 하고 학습을 시킨다 (학습 o)

#첫번째 실험
mobile_net_v2 = MobileNetV2()
mobile_net_v2.summary()

for layer in mobile_net_v2.layers[:,-1]:
    layer.trainable = True #traindata 를 학습 가능한 데이터로 만들수있다.


for layer in mobile_net_v2[:-1]:
    if 'kernel ' in layer.__dict__:

        kernel_shape = np.array(layer.get_weights()).shape
        layer.set_weights(np.random.normal(0,1,kernel_shape))

#1.독립변수를 생성해 주는 과정
#2. 같은 크기로 모든 이미지를 만들어주는 과정
#3. a모든 이미지를 minmax 할 과정


# 독립변수를 생성해 주는 과정
# 1. 모든 이미지를 같은 크기로 만들어 주는 과정
# 2. 모든 이미지를 min-max 시켜주는 과정


train_X = [] # 전처리를 마친 변수를 받는 빈 리스트 변수

# 모든 이미지의 수를 확인하고 반복문 활용
for i in range(len(label_text)):

    # 이미지 읽어들이기 (이미지 저장 경로를 확인해주세요.)
    img = cv2.imread('dataset/dogs/train/train/' + label_text['id'][i] + '.jpg')
    img = cv2.resize(img, dsize = (224,224)) #이미지 크기 통일

    # 이미지 정규화 과정
    img = img / 255.0 (min-max norm)

    train_x = append(img)

train_x = np.array(train_X)

print(train_x.shape)


#종속변수 생성 과정
unique_y = label_text['breed'].unique().tolist()

train_y = [unique_y.index(breed) for breed in label_text['bread']]
train_y = np.array(train_y)
#마지막 이전 계층
output_layer_2 = mobile_v2.layers[-2]

#신경망 출력계층의  None , 120
prediction = tf.keras.layers.Dense(units = 120,activation = 'softmax')(output_layer_2)

#input = 입력계층 , outputs = 출력 계층
model = tf.keras.Model(inputs = mobile_v2 ,outputs = prediction ) #가운데 은닉계층은 tf가 잡아온다 / but tf 가 어떻게 해주냐?
#그래서 함수형 방식은 마지막 계층이 어디에 위치하는지 알려줘야한다 즉 연결 시켜줘야 한다.
model.summary()

#compile 과정

model.compile(optimizers = tf.keras.optimizers.Adam(learning_rate = 0.0001),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

#텐서보드 를 위한 log 폴더 생성 과정
os.makedirs('/imageamp/log')

log_dir = '/imageamp/log'

#학습을 위한 코드
history = model.fit(train_x,train_y , epochs = 10 ,validation_split = 0.25,batch_size = 32,callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5) , tf.keras.callbacks.TensorBoard(log_dir = log_dir)])


#두번쨰 실험 : 절반의 계층 만 사용한다  freeze 방식

from tensorflow.keras.applications import MobileNetV2
import tensorflow as tf

mobilev2        = MobileNetV2()
output_layer_2  = mobilev2.layers[-2].output
predictions     = tf.keras.layers.Dense(120, activation='softmax')(output_layer_2)

model = tf.keras.Model(inputs=mobilev2.input, outputs=predictions)
model.summary()

#신경망 계층의 절반만 학습을 시키기 위한 코드
for layer in model.layers[:-20]:
    layer.trainable = False #처음 부터 중간 20 번쨰 계층 까지 를 학습 을 안시키기 위한 설정


for layer in model.layers[-20:]:
    layer.trainable = True #20번쨰 부터 마지막 번쨰 계층 까지 를 학습 을 시키기 위한 설정



model.compile(optimizers = 'sgd',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])


history = model.fit(train_x,train_y , epochs = 10 ,validation_split = 0.25,batch_size = 32,callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5) , tf.keras.callbacks.TensorBoard(log_dir = log_dir)])

#tensorboard 사용 방법
'''
1.아나콘다 프롬프트 에 실행 
2.현재 사용중인 가상 환경으로 이동한다.
conda activate '이름'
 
3. 가상 환경에서 텐서보드 실행 코드
### tensorboard --logdir=C 드라이브 의 경로 
### 여기서 log 파일까지의 경로는 여러분들의 경로로 잡아주시면 됩니다. 

4. 하단의 출력 되어진 localhost 를 복사 하여 웹에 붙여 넣어 준다.
'''

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')

plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()
model = tf.keras.models.load_model('save_model/ImageNet_dogs.h5')

img = plt.imread('dataset/dogs/test/test/001510bc8570bbeee98c8d80c8a95ec1.jpg')
img = np.resize(img,(224,224,3))
img = img / 255.0
img = np.expand_dims(img, axis=0)

prob = model.predict(img)


print("Predict Prob : \n",np.round(prob * 100,1))
plt.imshow(plt.imread('dataset/dogs/test/test/001510bc8570bbeee98c8d80c8a95ec1.jpg'))
print("Model Predict : Index-{}".format(np.argmax(prob)))